## Importaciones

In [1]:
# Montamos google drive a colab, en el caso de que trabajemos en ese entorno.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [98]:
import pandas as pd
import os
import numpy as np
import sys
import re
import pyarrow as pa
import pyarrow.parquet as pq
os.chdir('/content/drive/MyDrive/NoCountry_rendimiento_deportivo/notebooks') # si usamos colab y montamos drive
sys.path.append('/content/drive/MyDrive/NoCountry_rendimiento_deportivo/notebooks/resources.py') # si usamos colab y montamos drive
%load_ext autoreload
%autoreload 2
import resources as r

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
carpeta = os.listdir('../data')
carpeta

['appearances.csv.zip',
 'club_games.csv.zip',
 'clubs.csv',
 'competitions.csv',
 'game_events.csv.zip',
 'game_lineups.csv.zip',
 'games.csv.zip',
 'player_valuations.csv.zip',
 'players.csv.zip']

In [4]:
len(carpeta)

9

In [5]:
df_appearances = r.instanciar_data('../data/' + carpeta[0])

In [6]:
df_club_games = r.instanciar_data('../data/' + carpeta[1])

In [7]:
df_clubs = r.instanciar_data('../data/' + carpeta[2])

In [8]:
df_competitions = r.instanciar_data('../data/' + carpeta[3])

In [9]:
df_game_events = r.instanciar_data('../data/' + carpeta[4])

In [10]:
df_game_lineups= r.instanciar_data('../data/' + carpeta[5])

In [11]:
df_games = r.instanciar_data('../data/' + carpeta[6])

In [12]:
df_player_valuations = r.instanciar_data('../data/' + carpeta[7])

In [13]:
df_players = r.instanciar_data('../data/' + carpeta[8])

# <h1 align= 'center'><u>Appearances</u></h1>

## Informe Preliminar

### General

In [14]:
r.verificar_tipo_datos(df_appearances)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,appearance_id,[<class 'str'>],100.0,0.0,0
1,game_id,[<class 'int'>],100.0,0.0,0
2,player_id,[<class 'int'>],100.0,0.0,0
3,player_club_id,[<class 'int'>],100.0,0.0,0
4,player_current_club_id,[<class 'int'>],100.0,0.0,0
5,date,[<class 'str'>],100.0,0.0,0
6,player_name,"[<class 'str'>, <class 'float'>]",100.0,0.0,6
7,competition_id,[<class 'str'>],100.0,0.0,0
8,yellow_cards,[<class 'int'>],100.0,0.0,0
9,red_cards,[<class 'int'>],100.0,0.0,0


In [15]:
r.informe_dataframe(df_appearances)

INFORME PRELIMINAR SOBRE CARACTERISTICAS DEL DATASET:

--Dimensiones del DataFrame--
Filas: 1579515
Columnas: 13

--Numero de datos--
20533689

--Filas y Columnas--
Filas: muestra de indices-------> [0, 1, 2, 3, 4]  -----> Desde 0  Hasta 1579514
Columnas: ['appearance_id', 'game_id', 'player_id', 'player_club_id', 'player_current_club_id', 'date', 'player_name', 'competition_id', 'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played']

--Tipo de columnas--
appearance_id             object
game_id                    int64
player_id                  int64
player_club_id             int64
player_current_club_id     int64
date                      object
player_name               object
competition_id            object
yellow_cards               int64
red_cards                  int64
goals                      int64
assists                    int64
minutes_played             int64
dtype: object
--Cantidad de registros duplicados--
0



'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~oo~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

### Por columna

In [16]:
for columna in df_appearances.columns:
    r.informe_columna(df_appearances, columna)

INFORME PRELIMINAR SOBRE LA COLUMNA/FEATURE appearance_id:

--Numero de datos nulos--
0

--Cantidad de valores unicos en la columna--
1579515

--Valores unicos en la columna (Primeros 5 valores)--
['2231978_38004' '2233748_79232' '2234413_42792' '2234418_73333'
 '2234421_122011']

--Moda de la columna especificada--
Valor modal -----> 2231978_38004
Frecuencia acumulada ------> 1

--Distribucion de frecuencias (primeros valores con mayor cantidad de frecuencias)--
 appearance_id
2231978_38004     1
3394883_105695    1
3412937_284732    1
Name: count, dtype: int64

------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------
INFORME PRELIMINAR SOBRE LA COLUMNA/FEATURE game_id:

--Numero de datos nulos--
0

--Valores unicos en la columna--
Ej: [2231978 2233748 2234413 2234418 2234421]  -----> Desde 2231978  Has

 ## ETL

In [17]:
df_appearances.head()

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,2231978_38004,2231978,38004,853,235,2012-07-03,Aurélien Joachim,CLQ,0,0,2,0,90
1,2233748_79232,2233748,79232,8841,2698,2012-07-05,Ruslan Abyshov,ELQ,0,0,0,0,90
2,2234413_42792,2234413,42792,6251,465,2012-07-05,Sander Puri,ELQ,0,0,0,0,45
3,2234418_73333,2234418,73333,1274,6646,2012-07-05,Vegar Hedenstad,ELQ,0,0,0,0,90
4,2234421_122011,2234421,122011,195,3008,2012-07-05,Markus Henriksen,ELQ,0,0,0,1,90


In [18]:
df_appearances.isna().sum()

appearance_id             0
game_id                   0
player_id                 0
player_club_id            0
player_current_club_id    0
date                      0
player_name               6
competition_id            0
yellow_cards              0
red_cards                 0
goals                     0
assists                   0
minutes_played            0
dtype: int64

In [19]:
df_appearances['player_name'] = df_appearances['player_name'].fillna('None') # se rellena valores nan por None

In [20]:
df_appearances['date'] = pd.to_datetime(df_appearances['date']) # se cambia el tipo de dato de la columna

* Se reemplazo los valores nan por una etiqueta 'None' en la columna 'player_name'.
* Se cambio el tipo de dato de 'object' a 'datetime'.

# <h1 align= 'center'><u>Clubs</u></h1>

## Informe Preliminar

### General

In [21]:
r.verificar_tipo_datos(df_clubs)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,club_id,[<class 'int'>],100.00,0.00,0
1,club_code,[<class 'str'>],100.00,0.00,0
2,name,[<class 'str'>],100.00,0.00,0
3,domestic_competition_id,[<class 'str'>],100.00,0.00,0
4,total_market_value,[<class 'float'>],0.00,100.00,426
5,squad_size,[<class 'int'>],100.00,0.00,0
6,average_age,[<class 'float'>],91.08,8.92,38
7,foreigners_number,[<class 'int'>],100.00,0.00,0
8,foreigners_percentage,[<class 'float'>],88.73,11.27,48
9,national_team_players,[<class 'int'>],100.00,0.00,0


In [22]:
r.informe_dataframe(df_clubs)

INFORME PRELIMINAR SOBRE CARACTERISTICAS DEL DATASET:

--Dimensiones del DataFrame--
Filas: 426
Columnas: 17

--Numero de datos--
6304

--Filas y Columnas--
Filas: muestra de indices-------> [0, 1, 2, 3, 4]  -----> Desde 0  Hasta 425
Columnas: ['club_id', 'club_code', 'name', 'domestic_competition_id', 'total_market_value', 'squad_size', 'average_age', 'foreigners_number', 'foreigners_percentage', 'national_team_players', 'stadium_name', 'stadium_seats', 'net_transfer_record', 'coach_name', 'last_season', 'filename', 'url']

--Tipo de columnas--
club_id                      int64
club_code                   object
name                        object
domestic_competition_id     object
total_market_value         float64
squad_size                   int64
average_age                float64
foreigners_number            int64
foreigners_percentage      float64
national_team_players        int64
stadium_name                object
stadium_seats                int64
net_transfer_record         

'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~oo~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

### Por columna

In [23]:
for columna in df_clubs.columns:
    r.informe_columna(df_clubs, columna)

INFORME PRELIMINAR SOBRE LA COLUMNA/FEATURE club_id:

--Numero de datos nulos--
0

--Valores unicos en la columna--
Ej: [  105 11127   114    12   148]  -----> Desde 105  Hasta 984

--Estadisticos Principales de la columna--
Media: 5314.53
Desviacion Estandar: 11752.01
Primer cuartil: 421.0
Mediana: 1139.5
Tercer cuartil: 3415.75

--Valores extremos--
Valor maximo: 83678
Valor minimo: 3

--Distribucion de frecuencias (primeros valores con mayor cantidad de frecuencias)--
 club_id
105     1
1416    1
1387    1
Name: count, dtype: int64

--Valor maximo y minimo--
Maximo: 83678
Minimo: 3

------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------
INFORME PRELIMINAR SOBRE LA COLUMNA/FEATURE club_code:

--Numero de datos nulos--
0

--Cantidad de valores unicos en la columna--
426

--Valores unicos en la columna

## ETL

In [24]:
df_clubs.head()

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
0,105,sv-darmstadt-98,Sportverein Darmstadt 1898 e. V.,L1,NaN,31,26.7,11,35.5,1,Merck-Stadion am Böllenfalltor,17810,€-1.48m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/sv-darmstadt-9...
1,11127,ural-ekaterinburg,FK Ural Yekaterinburg,RU1,NaN,27,28.1,15,55.6,6,Yekaterinburg Arena,23000,€-895k,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/ural-ekaterinb...
2,114,besiktas-istanbul,Beşiktaş Jimnastik Kulübü,TR1,NaN,37,26.3,17,45.9,14,Tüpraş Stadyumu,42445,€-26.00m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/besiktas-istan...
3,12,as-rom,Associazione Sportiva Roma,IT1,NaN,26,27.1,18,69.2,16,Olimpico di Roma,73261,+€63.20m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/as-rom/startse...
4,148,tottenham-hotspur,Tottenham Hotspur Football Club,GB1,NaN,26,25.7,20,76.9,20,Tottenham Hotspur Stadium,62850,€-151.40m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/tottenham-hots...


In [25]:
df_clubs.isna().sum()

club_id                      0
club_code                    0
name                         0
domestic_competition_id      0
total_market_value         426
squad_size                   0
average_age                 38
foreigners_number            0
foreigners_percentage       48
national_team_players        0
stadium_name                 0
stadium_seats                0
net_transfer_record          0
coach_name                 426
last_season                  0
filename                     0
url                          0
dtype: int64

In [26]:
# Eliminar columnas con mucho porcentaje de valores nan o con data que consideramos irrelevante
df_clubs.drop(columns= ['total_market_value', 'coach_name','club_code', 'filename', 'url'], inplace= True)

In [27]:
#Elimina la palabra simbolos, letras y transforma a valor flotante columna 'net_transfer_record'
df_clubs['net_transfer_record'] = df_clubs['net_transfer_record'].str.replace('m', '').str.replace('€-', '').str.replace('+€', '').str.replace('k', '').str.replace('+-', '').astype(float)


In [28]:
df_clubs['average_age'] = df_clubs['average_age'].fillna(df_clubs['average_age'].mean())
df_clubs['foreigners_percentage'] = df_clubs['foreigners_percentage'].fillna(df_clubs['foreigners_percentage'].mean())

* Se eliminaron columnas con gran porcentaje de valores nulos o faltantes ('total_market_value', 'coach_name').
* Se eliminaron columnas que contenían data que consideramos irrelevante ('club_code', 'filename', 'url')
* Se normalizó y formateó los datos de las columnas 'average_age' y 'foreigners_percentage'

# <h1 align= 'center'><u>Club Games</u></h1>

## Informe Preliminar

### General

In [29]:
r.verificar_tipo_datos(df_club_games)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,game_id,[<class 'int'>],100.00,0.00,0
1,club_id,[<class 'int'>],100.00,0.00,0
2,own_goals,[<class 'int'>],100.00,0.00,0
3,own_position,[<class 'float'>],70.60,29.40,40086
4,own_manager_name,"[<class 'str'>, <class 'float'>]",98.91,1.09,1490
5,opponent_id,[<class 'int'>],100.00,0.00,0
6,opponent_goals,[<class 'int'>],100.00,0.00,0
7,opponent_position,[<class 'float'>],70.60,29.40,40086
8,opponent_manager_name,"[<class 'str'>, <class 'float'>]",98.91,1.09,1490
9,hosting,[<class 'str'>],100.00,0.00,0


In [30]:
r.informe_dataframe(df_club_games)

INFORME PRELIMINAR SOBRE CARACTERISTICAS DEL DATASET:

--Dimensiones del DataFrame--
Filas: 136344
Columnas: 11

--Numero de datos--
1416632

--Filas y Columnas--
Filas: muestra de indices-------> [0, 1, 2, 3, 4]  -----> Desde 0  Hasta 136343
Columnas: ['game_id', 'club_id', 'own_goals', 'own_position', 'own_manager_name', 'opponent_id', 'opponent_goals', 'opponent_position', 'opponent_manager_name', 'hosting', 'is_win']

--Tipo de columnas--
game_id                    int64
club_id                    int64
own_goals                  int64
own_position             float64
own_manager_name          object
opponent_id                int64
opponent_goals             int64
opponent_position        float64
opponent_manager_name     object
hosting                   object
is_win                     int64
dtype: object
--Cantidad de registros duplicados--
0



'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~oo~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

### Por columna

In [31]:
for columna in df_club_games.columns:
    r.informe_columna(df_club_games, columna)

INFORME PRELIMINAR SOBRE LA COLUMNA/FEATURE game_id:

--Numero de datos nulos--
0

--Valores unicos en la columna--
Ej: [2320450 2320460 2320472 2321044 2321060]  -----> Desde 2320450  Hasta 3520159

--Estadisticos Principales de la columna--
Media: 3063156.19
Desviacion Estandar: 590641.23
Primer cuartil: 2559106.75
Mediana: 3047452.5
Tercer cuartil: 3583517.25

--Valores extremos--
Valor maximo: 4340617
Valor minimo: 2211607

--Distribucion de frecuencias (primeros valores con mayor cantidad de frecuencias)--
 game_id
2320450    2
3690536    2
3631966    2
Name: count, dtype: int64

--Valor maximo y minimo--
Maximo: 4340617
Minimo: 2211607

------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------
INFORME PRELIMINAR SOBRE LA COLUMNA/FEATURE club_id:

--Numero de datos nulos--
0

--Valores unicos en la c

Notas

* En la columna game_id la cantidad de frecuencias por cada valor debe ser siempre 2, caso contraio faltan datos o sobran datos, o hay eroores en los datos
* el que algunos DTs tengan muchos partidos o algunos equipos tengan mas partidos que otros, da cuenta de su calidad relativa ya que significa que ingresaron a mas competencias, o que llegaron mas lejos en esas competencias.
* en la columna is_win se sobreentiende que los valores 0 y 1 que son los valores unicos, hacen referencia a True y False (False== 0, True == 1)

## ETL

In [32]:
df_club_games.head()

,game_id,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win
0,2320450,1468,0,NaN,Holger Bachthaler,24,2,NaN,Armin Veh,Home,0
1,2320460,1,3,NaN,Jürgen Luginger,86,1,NaN,Robin Dutt,Home,1
2,2320472,2036,4,NaN,Frank Schmidt,72,5,NaN,Alexander Schmidt,Home,0
3,2321044,16,2,1.0,Jürgen Klopp,23,1,15.0,Torsten Lieberknecht,Home,1
4,2321060,23,0,18.0,Torsten Lieberknecht,24,2,11.0,Armin Veh,Home,0


In [33]:
# reemplazamos los valores nan
df_club_games['own_position'] = df_club_games['own_position'].fillna(-1).astype(int)
df_club_games['opponent_position'] = df_club_games['opponent_position'].fillna(-1).astype(int)
df_club_games['opponent_manager_name'] = df_club_games['opponent_manager_name'].fillna('None').astype(str)
df_club_games['own_manager_name'] = df_club_games['own_manager_name'].fillna('None').astype(str)



In [34]:
df_club_games['own_goals'].value_counts()

own_goals
1     43676
0     37490
2     29470
3     14748
4      6301
5      2747
6      1099
7       435
8       187
9       100
10       33
11       17
12       16
13        7
14        7
15        7
16        3
19        1
Name: count, dtype: int64

* Se reemplazan los valores nan de las columnas ('own_position', 'opponent_position') por el valor -1, y las columnas ('own_manager_name', 'opponent_manager_name') por 'None'.
*existen algunos valores muy extremos en la columna own_goals (y tambien en opponent_goals)

# <h1 align= 'center'><u>Competitions</u></h1>

## Informe Preliminar

### General

In [35]:
r.verificar_tipo_datos(df_competitions)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,competition_id,[<class 'str'>],100.00,0.00,0
1,competition_code,[<class 'str'>],100.00,0.00,0
2,name,[<class 'str'>],100.00,0.00,0
3,sub_type,[<class 'str'>],100.00,0.00,0
4,type,[<class 'str'>],100.00,0.00,0
5,country_id,[<class 'int'>],100.00,0.00,0
6,country_name,"[<class 'str'>, <class 'float'>]",83.72,16.28,7
7,domestic_league_code,"[<class 'str'>, <class 'float'>]",83.72,16.28,7
8,confederation,[<class 'str'>],100.00,0.00,0
9,url,[<class 'str'>],100.00,0.00,0


In [36]:
r.informe_dataframe(df_competitions)

INFORME PRELIMINAR SOBRE CARACTERISTICAS DEL DATASET:

--Dimensiones del DataFrame--
Filas: 43
Columnas: 11

--Numero de datos--
459

--Filas y Columnas--
Filas: muestra de indices-------> [0, 1, 2, 3, 4]  -----> Desde 0  Hasta 42
Columnas: ['competition_id', 'competition_code', 'name', 'sub_type', 'type', 'country_id', 'country_name', 'domestic_league_code', 'confederation', 'url', 'is_major_national_league']

--Tipo de columnas--
competition_id              object
competition_code            object
name                        object
sub_type                    object
type                        object
country_id                   int64
country_name                object
domestic_league_code        object
confederation               object
url                         object
is_major_national_league      bool
dtype: object
--Cantidad de registros duplicados--
0



'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~oo~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

### Por columna

In [37]:
for columna in df_competitions.columns:
    r.informe_columna(df_competitions, columna)

INFORME PRELIMINAR SOBRE LA COLUMNA/FEATURE competition_id:

--Numero de datos nulos--
0

--Cantidad de valores unicos en la columna--
43

--Valores unicos en la columna (Primeros 5 valores)--
['CIT' 'NLSC' 'GRP' 'POSU' 'RUSS']

--Moda de la columna especificada--
Valor modal -----> CIT
Frecuencia acumulada ------> 1

--Distribucion de frecuencias (primeros valores con mayor cantidad de frecuencias)--
 competition_id
CIT     1
L1      1
UKR1    1
Name: count, dtype: int64

------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------
INFORME PRELIMINAR SOBRE LA COLUMNA/FEATURE competition_code:

--Numero de datos nulos--
0

--Cantidad de valores unicos en la columna--
42

--Valores unicos en la columna (Primeros 5 valores)--
['italy-cup' 'johan-cruijff-schaal' 'kypello-elladas'
 'supertaca-candido-de-oliveira

Notas

* Revisar en la columna 'name' el valor de premier-liga, ya que presenta una frecuencia acumulada de 2 cuando lo normal seria 1
* Valores -1 de columna country_id representan las competiciones que no son de un solo pais en especifico (como la UEFA)

In [38]:
df_competitions[df_competitions['name'] == 'premier-liga']

,competition_id,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url,is_major_national_league
24,UKR1,premier-liga,premier-liga,first_tier,domestic_league,177,Ukraine,UKR1,europa,https://www.transfermarkt.co.uk/premier-liga/s...,False
33,RU1,premier-liga,premier-liga,first_tier,domestic_league,141,Russia,RU1,europa,https://www.transfermarkt.co.uk/premier-liga/s...,False


## ETL

In [39]:
df_competitions.head()

,competition_id,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url,is_major_national_league
0,CIT,italy-cup,italy-cup,domestic_cup,domestic_cup,75,Italy,IT1,europa,https://www.transfermarkt.co.uk/italy-cup/star...,False
1,NLSC,johan-cruijff-schaal,johan-cruijff-schaal,domestic_super_cup,other,122,Netherlands,NL1,europa,https://www.transfermarkt.co.uk/johan-cruijff-...,False
2,GRP,kypello-elladas,kypello-elladas,domestic_cup,domestic_cup,56,Greece,GR1,europa,https://www.transfermarkt.co.uk/kypello-ellada...,False
3,POSU,supertaca-candido-de-oliveira,supertaca-candido-de-oliveira,domestic_super_cup,other,136,Portugal,PO1,europa,https://www.transfermarkt.co.uk/supertaca-cand...,False
4,RUSS,russian-super-cup,russian-super-cup,domestic_super_cup,other,141,Russia,RU1,europa,https://www.transfermarkt.co.uk/russian-super-...,False


In [40]:
# veamos que registro presentan valores nulos en la columna country_name
df_competitions[df_competitions['country_name'].isna()]

,competition_id,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url,is_major_national_league
6,USC,uefa-super-cup,uefa-super-cup,uefa_super_cup,other,-1,NaN,NaN,europa,https://www.transfermarkt.co.uk/uefa-super-cup...,False
8,EL,europa-league,europa-league,europa_league,international_cup,-1,NaN,NaN,europa,https://www.transfermarkt.co.uk/europa-league/...,False
17,ELQ,europa-league-qualifikation,europa-league-qualifikation,europa_league_qualifying,international_cup,-1,NaN,NaN,europa,https://www.transfermarkt.co.uk/europa-league-...,False
20,ECLQ,uefa-europa-conference-league-qualifikation,uefa-europa-conference-league-qualifikation,uefa_europa_conference_league_qualifiers,international_cup,-1,NaN,NaN,europa,https://www.transfermarkt.co.uk/uefa-europa-co...,False
26,CL,uefa-champions-league,uefa-champions-league,uefa_champions_league,international_cup,-1,NaN,NaN,europa,https://www.transfermarkt.co.uk/uefa-champions...,False
31,KLUB,fifa-klub-wm,fifa-klub-wm,fifa_club_world_cup,other,-1,NaN,NaN,europa,https://www.transfermarkt.co.uk/fifa-klub-wm/s...,False
37,CLQ,uefa-champions-league-qualifikation,uefa-champions-league-qualifikation,uefa_champions_league_qualifying,international_cup,-1,NaN,NaN,europa,https://www.transfermarkt.co.uk/uefa-champions...,False


In [41]:
# reemplazamos los valores nan por una etiqueta None, ya que esas competencias no pertenecen a un solo pais

df_competitions['country_name'] = df_competitions['country_name'].fillna('None').astype(str)
df_competitions['domestic_league_code'] = df_competitions['domestic_league_code'].fillna('None').astype(str)

In [42]:
# eliminamos columna que no contiene data relevante ni utilizable para nuestro proyecto
df_competitions.drop(columns= 'url', inplace= True)

In [43]:
df_competitions['is_major_national_league'] = df_competitions['is_major_national_league'].astype(str)

# <h1 align= 'center'><u>Game Lineups</u></h1>

## Informe Preliminar

### General

In [95]:
r.verificar_tipo_datos(df_game_lineups)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,game_lineups_id,[<class 'str'>],100.0,0.0,0
1,date,[<class 'pandas._libs.tslibs.timestamps.Timest...,100.0,0.0,0
2,game_id,[<class 'int'>],100.0,0.0,0
3,player_id,[<class 'int'>],100.0,0.0,0
4,club_id,[<class 'int'>],100.0,0.0,0
5,player_name,[<class 'str'>],100.0,0.0,0
6,type,[<class 'str'>],100.0,0.0,0
7,position,[<class 'str'>],100.0,0.0,0
8,number,[<class 'int'>],100.0,0.0,0
9,team_captain,[<class 'int'>],100.0,0.0,0


In [45]:
r.informe_dataframe(df_game_lineups)

INFORME PRELIMINAR SOBRE CARACTERISTICAS DEL DATASET:

--Dimensiones del DataFrame--
Filas: 2191911
Columnas: 10

--Numero de datos--
21919107

--Filas y Columnas--
Filas: muestra de indices-------> [0, 1, 2, 3, 4]  -----> Desde 0  Hasta 2191910
Columnas: ['game_lineups_id', 'date', 'game_id', 'player_id', 'club_id', 'player_name', 'type', 'position', 'number', 'team_captain']

--Tipo de columnas--
game_lineups_id    object
date               object
game_id             int64
player_id           int64
club_id             int64
player_name        object
type               object
position           object
number             object
team_captain        int64
dtype: object
--Cantidad de registros duplicados--
0



'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~oo~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

### Por columna

In [46]:
for columna in df_game_lineups.columns:
    r.informe_columna(df_game_lineups, columna)

INFORME PRELIMINAR SOBRE LA COLUMNA/FEATURE game_lineups_id:

--Numero de datos nulos--
0

--Cantidad de valores unicos en la columna--
2191911

--Valores unicos en la columna (Primeros 5 valores)--
['b2dbe01c3656b06c8e23e9de714e26bb' 'b50a3ec6d52fd1490aab42042ac4f738'
 '7d890e6d0ff8af84b065839966a0ec81' '8c355268678b9bbc7084221b1f0fde36'
 '76193074d549e5fdce4cdcbba0d66247']

--Moda de la columna especificada--
Valor modal -----> b2dbe01c3656b06c8e23e9de714e26bb
Frecuencia acumulada ------> 1

--Distribucion de frecuencias (primeros valores con mayor cantidad de frecuencias)--
 game_lineups_id
b2dbe01c3656b06c8e23e9de714e26bb    1
268011a34a45c44e8283bf964f936f3f    1
6760952f2960de1e63c90051aed889e9    1
Name: count, dtype: int64

------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------
INFORME PRELIMIN

Notas

* Se puede averiguar la posicion de los jugadores en la columna 'position' y rellenarla
* los valores 0 y 1 en team_captain hacen referencia a True y False (False == 0, True == 1)

## ETL

In [94]:
df_game_lineups.head()

,game_lineups_id,date,game_id,player_id,club_id,player_name,type,position,number,team_captain
0,b2dbe01c3656b06c8e23e9de714e26bb,2013-07-27,2317258,1443,610,Christian Poulsen,substitutes,Defensive Midfield,5,0
1,b50a3ec6d52fd1490aab42042ac4f738,2013-07-27,2317258,5017,610,Niklas Moisander,starting_lineup,Centre-Back,4,0
2,7d890e6d0ff8af84b065839966a0ec81,2013-07-27,2317258,9602,1090,Maarten Martens,substitutes,Left Winger,11,0
3,8c355268678b9bbc7084221b1f0fde36,2013-07-27,2317258,12282,610,Daley Blind,starting_lineup,Left-Back,17,0
4,76193074d549e5fdce4cdcbba0d66247,2013-07-27,2317258,25427,1090,Roy Beerens,starting_lineup,Right Winger,23,0


In [93]:
# Averiguamos los valores nan de la columna 'position'
df_game_lineups[df_game_lineups['position'].isna()]


,game_lineups_id,date,game_id,player_id,club_id,player_name,type,position,number,team_captain


In [90]:
# cambiamos el tipo de dato de la columna date a uno datetime

df_game_lineups['date'] = pd.to_datetime(df_game_lineups['date'])

In [92]:
# normalizamos los datos de la columna 'number' y transformamos la columna a tipo int
df_game_lineups['number'] = df_game_lineups['number'].apply(lambda x: ''.join(re.findall(r'\d+', x.strip())))
df_game_lineups['number'] = df_game_lineups['number'].replace('', '-1')
df_game_lineups['number'] = df_game_lineups['number'].astype(int)

In [50]:
# reemplazamos los valores nan por los valores correspondientes
df_game_lineups['position'].loc[242898] = 'midfield'
df_game_lineups['position'].loc[501768] = 'Attack'
df_game_lineups['position'].loc[501771] = 'None'

* se rellenaron los valores nan de 'position con los valores:
  - Shaun Dunion == midfield
  - Jimmy Theelen == Attack
  - Bart Zeevenhoven == None
* Se cambio el tipo de dato de la columna date de str a datetime.
* Normalizamos los datos de la columna 'number' y transformamos la columna a tipo int

# <h1 align= 'center'><u>Games</u></h1>

## Informe Preliminar

### General

In [51]:
r.verificar_tipo_datos(df_games)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,game_id,[<class 'int'>],100.00,0.00,0
1,competition_id,[<class 'str'>],100.00,0.00,0
2,season,[<class 'int'>],100.00,0.00,0
3,round,[<class 'str'>],100.00,0.00,0
4,date,[<class 'str'>],100.00,0.00,0
5,home_club_id,[<class 'int'>],100.00,0.00,0
6,away_club_id,[<class 'int'>],100.00,0.00,0
7,home_club_goals,[<class 'int'>],100.00,0.00,0
8,away_club_goals,[<class 'int'>],100.00,0.00,0
9,home_club_position,[<class 'float'>],70.60,29.40,20043


In [52]:
r.informe_dataframe(df_games)

INFORME PRELIMINAR SOBRE CARACTERISTICAS DEL DATASET:

--Dimensiones del DataFrame--
Filas: 68172
Columnas: 23

--Numero de datos--
1481365

--Filas y Columnas--
Filas: muestra de indices-------> [0, 1, 2, 3, 4]  -----> Desde 0  Hasta 68171
Columnas: ['game_id', 'competition_id', 'season', 'round', 'date', 'home_club_id', 'away_club_id', 'home_club_goals', 'away_club_goals', 'home_club_position', 'away_club_position', 'home_club_manager_name', 'away_club_manager_name', 'stadium', 'attendance', 'referee', 'url', 'home_club_formation', 'away_club_formation', 'home_club_name', 'away_club_name', 'aggregate', 'competition_type']

--Tipo de columnas--
game_id                     int64
competition_id             object
season                      int64
round                      object
date                       object
home_club_id                int64
away_club_id                int64
home_club_goals             int64
away_club_goals             int64
home_club_position        float64
away_c

'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~oo~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

### Por columna

In [53]:
for columna in df_games.columns:
    r.informe_columna(df_games, columna)

INFORME PRELIMINAR SOBRE LA COLUMNA/FEATURE game_id:

--Numero de datos nulos--
0

--Valores unicos en la columna--
Ej: [2321044 2321060 2321086 2321152 2321205]  -----> Desde 2321044  Hasta 4233201

--Estadisticos Principales de la columna--
Media: 3063156.19
Desviacion Estandar: 590643.39
Primer cuartil: 2559106.75
Mediana: 3047452.5
Tercer cuartil: 3583517.25

--Valores extremos--
Valor maximo: 4340617
Valor minimo: 2211607

--Distribucion de frecuencias (primeros valores con mayor cantidad de frecuencias)--
 game_id
2321044    1
3058658    1
3050447    1
Name: count, dtype: int64

--Valor maximo y minimo--
Maximo: 4340617
Minimo: 2211607

------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------
INFORME PRELIMINAR SOBRE LA COLUMNA/FEATURE competition_id:

--Numero de datos nulos--
0

--Cantidad de val

## ETL

In [54]:
df_games.head()

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
0,2321044,L1,2013,2. Matchday,2013-08-18,16,23,2,1,1.0,...,SIGNAL IDUNA PARK,80200.0,Peter Sippel,https://www.transfermarkt.co.uk/borussia-dortm...,4-2-3-1,4-3-2-1,Borussia Dortmund,Eintracht Braunschweig,2:1,domestic_league
1,2321060,L1,2013,3. Matchday,2013-08-25,23,24,0,2,18.0,...,EINTRACHT-Stadion,23325.0,Wolfgang Stark,https://www.transfermarkt.co.uk/eintracht-brau...,4-3-2-1,4-2-3-1,Eintracht Braunschweig,Eintracht Frankfurt Fußball AG,0:2,domestic_league
2,2321086,L1,2013,6. Matchday,2013-09-21,4,16,1,1,15.0,...,Max-Morlock-Stadion,50000.0,Knut Kircher,https://www.transfermarkt.co.uk/1-fc-nuremberg...,4-2-3-1,4-2-3-1,1.FC Nuremberg,Borussia Dortmund,1:1,domestic_league
3,2321152,L1,2013,11. Matchday,2013-11-02,44,33,0,2,7.0,...,Olympiastadion Berlin,69277.0,Günter Perl,https://www.transfermarkt.co.uk/hertha-bsc_fc-...,4-2-3-1,4-2-3-1,Hertha BSC,FC Schalke 04,0:2,domestic_league
4,2321205,L1,2013,17. Matchday,2013-12-21,41,39,2,3,14.0,...,Volksparkstadion,50000.0,Bastian Dankert,https://www.transfermarkt.co.uk/hamburger-sv_1...,4-2-3-1,4-4-2 Diamond,Hamburger SV,1. Fußball- und Sportverein Mainz 05,2:3,domestic_league


In [55]:
# rellenamos valores nan
df_games['home_club_position'].fillna(-1, inplace= True)
df_games['away_club_position'].fillna(-1, inplace= True)
df_games['home_club_manager_name'].fillna('None', inplace= True)
df_games['away_club_manager_name'].fillna('None', inplace= True)
df_games['stadium'].fillna('None', inplace= True)
df_games['attendance'].fillna(-1, inplace= True)
df_games['referee'].fillna('None', inplace= True)
df_games['home_club_formation'].fillna('None', inplace= True)
df_games['away_club_formation'].fillna('None', inplace= True)
df_games['home_club_name'].fillna('None', inplace= True)
df_games['away_club_name'].fillna('None', inplace= True)

In [56]:
# cambiamos el tipo de dato de las columnas que lo requieren

df_games['date'] = pd.to_datetime(df_games['date'])
df_games['home_club_position'] = df_games['home_club_position'].astype(int)
df_games['away_club_position'] = df_games['away_club_position'].astype(int)
df_games['attendance'] = df_games['attendance'].astype(int)

In [57]:
# eliminamos columna url porque no nos brinda informacion reelevante para el proyecto

df_games.drop(columns= 'url', inplace= True)

# <h1 align= 'center'><u>Game Events</u></h1>

## Informe preliminar

### General

In [58]:
r.verificar_tipo_datos(df_game_events)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,game_event_id,[<class 'str'>],100.00,0.00,0
1,date,[<class 'str'>],100.00,0.00,0
2,game_id,[<class 'int'>],100.00,0.00,0
3,minute,[<class 'int'>],100.00,0.00,0
4,type,[<class 'str'>],100.00,0.00,0
5,club_id,[<class 'int'>],100.00,0.00,0
6,player_id,[<class 'int'>],100.00,0.00,0
7,description,"[<class 'str'>, <class 'float'>]",56.67,43.33,319471
8,player_in_id,[<class 'float'>],60.52,39.48,291086
9,player_assist_id,[<class 'float'>],6.61,93.39,688600


In [59]:
r.informe_dataframe(df_game_events)

INFORME PRELIMINAR SOBRE CARACTERISTICAS DEL DATASET:

--Dimensiones del DataFrame--
Filas: 737339
Columnas: 10

--Numero de datos--
6074233

--Filas y Columnas--
Filas: muestra de indices-------> [0, 1, 2, 3, 4]  -----> Desde 0  Hasta 737338
Columnas: ['game_event_id', 'date', 'game_id', 'minute', 'type', 'club_id', 'player_id', 'description', 'player_in_id', 'player_assist_id']

--Tipo de columnas--
game_event_id        object
date                 object
game_id               int64
minute                int64
type                 object
club_id               int64
player_id             int64
description          object
player_in_id        float64
player_assist_id    float64
dtype: object
--Cantidad de registros duplicados--
0



'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~oo~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

### Por columna

In [60]:
for columna in df_game_events.columns:
    r.informe_columna(df_game_events, columna)

INFORME PRELIMINAR SOBRE LA COLUMNA/FEATURE game_event_id:

--Numero de datos nulos--
0

--Cantidad de valores unicos en la columna--
737339

--Valores unicos en la columna (Primeros 5 valores)--
['2f41da30c471492e7d4a984951671677' 'a72f7186d132775f234d3e2f7bc0ed5b'
 'b2d721eaed4692a5c59a92323689ef18' 'aef768899cedac0c9a650980219075a2'
 '5d6d9533023057b6619ecd145a038bbe']

--Moda de la columna especificada--
Valor modal -----> 2f41da30c471492e7d4a984951671677
Frecuencia acumulada ------> 1

--Distribucion de frecuencias (primeros valores con mayor cantidad de frecuencias)--
 game_event_id
2f41da30c471492e7d4a984951671677    1
ca9cb564fc129d2aff57c96df41485cb    1
c7183f6fd6797793af705e3bae4e9d69    1
Name: count, dtype: int64

------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------
INFORME PRELIMINAR SO

## ETL

In [61]:
df_game_events.head()

,game_event_id,date,game_id,minute,type,club_id,player_id,description,player_in_id,player_assist_id
0,2f41da30c471492e7d4a984951671677,2012-08-05,2211607,77,Cards,610,4425,"1. Yellow card , Mass confrontation",NaN,NaN
1,a72f7186d132775f234d3e2f7bc0ed5b,2012-08-05,2211607,77,Cards,383,33210,"1. Yellow card , Mass confrontation",NaN,NaN
2,b2d721eaed4692a5c59a92323689ef18,2012-08-05,2211607,3,Goals,383,36500,", Header, 1. Tournament Goal Assist: , Corner,...",NaN,56416.0
3,aef768899cedac0c9a650980219075a2,2012-08-05,2211607,53,Goals,383,36500,", Right-footed shot, 2. Tournament Goal Assist...",NaN,146258.0
4,5d6d9533023057b6619ecd145a038bbe,2012-08-05,2211607,74,Substitutions,383,36500,", Not reported",49499.0,NaN


In [62]:
# rellenamos valores nan y cambiamos el tipo de dato de las columnas

df_game_events['player_in_id'] = df_game_events['player_in_id'].fillna(-1).astype(int)
df_game_events['player_assist_id'] = df_game_events['player_assist_id'].fillna(-1).astype(int)

In [63]:
# Cambiamos el tipo de dato de la columna date por datetime

df_game_events['date'] = pd.to_datetime(df_game_events['date'])

In [64]:
# Eliminamos la columna description porque consideramos que esa data la proporciona la columna type por lo que es redundante

df_game_events.drop(columns= 'description', inplace= True)

Notas

* Es importante señalar que este dataframe hace referencia a eventos ocurridos en un determinado partido
* en player_in_id el valor -1 es porque el evento no es una substitucion, en player_assist_id el valor -1 es porque el evento no es un gol

# <h1 align= 'center'><u>Players</u></h1>

## Informe Preliminar

### General

In [65]:
r.verificar_tipo_datos(df_players)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,player_id,[<class 'int'>],100.00,0.00,0
1,first_name,"[<class 'str'>, <class 'float'>]",93.48,6.52,1989
2,last_name,[<class 'str'>],100.00,0.00,0
3,name,[<class 'str'>],100.00,0.00,0
4,last_season,[<class 'int'>],100.00,0.00,0
5,current_club_id,[<class 'int'>],100.00,0.00,0
6,player_code,[<class 'str'>],100.00,0.00,0
7,country_of_birth,"[<class 'str'>, <class 'float'>]",91.11,8.89,2712
8,city_of_birth,"[<class 'str'>, <class 'float'>]",92.83,7.17,2189
9,country_of_citizenship,"[<class 'str'>, <class 'float'>]",98.10,1.90,579


In [66]:
r.informe_dataframe(df_players)

INFORME PRELIMINAR SOBRE CARACTERISTICAS DEL DATASET:

--Dimensiones del DataFrame--
Filas: 30518
Columnas: 23

--Numero de datos--
660818

--Filas y Columnas--
Filas: muestra de indices-------> [0, 1, 2, 3, 4]  -----> Desde 0  Hasta 30517
Columnas: ['player_id', 'first_name', 'last_name', 'name', 'last_season', 'current_club_id', 'player_code', 'country_of_birth', 'city_of_birth', 'country_of_citizenship', 'date_of_birth', 'sub_position', 'position', 'foot', 'height_in_cm', 'contract_expiration_date', 'agent_name', 'image_url', 'url', 'current_club_domestic_competition_id', 'current_club_name', 'market_value_in_eur', 'highest_market_value_in_eur']

--Tipo de columnas--
player_id                                 int64
first_name                               object
last_name                                object
name                                     object
last_season                               int64
current_club_id                           int64
player_code                      

'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~oo~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

### Por columna

In [67]:
for columna in df_players.columns:
    r.informe_columna(df_players, columna)

INFORME PRELIMINAR SOBRE LA COLUMNA/FEATURE player_id:

--Numero de datos nulos--
0

--Valores unicos en la columna--
Ej: [10 26 65 77 80]  -----> Desde 10  Hasta 1240762

--Estadisticos Principales de la columna--
Media: 314777.71
Desviacion Estandar: 253725.34
Primer cuartil: 96015.25
Mediana: 259886.5
Tercer cuartil: 470831.75

--Valores extremos--
Valor maximo: 1240762
Valor minimo: 10

--Distribucion de frecuencias (primeros valores con mayor cantidad de frecuencias)--
 player_id
10        1
387110    1
387218    1
Name: count, dtype: int64

--Valor maximo y minimo--
Maximo: 1240762
Minimo: 10

------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------
INFORME PRELIMINAR SOBRE LA COLUMNA/FEATURE first_name:

--Numero de datos nulos--
1989

--Cantidad de valores unicos en la columna--
6603

--Valores u

## ETL

In [68]:
df_players.head()

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,right,184.0,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0
1,26,Roman,Weidenfeller,Roman Weidenfeller,2017,16,roman-weidenfeller,Germany,Diez,Germany,...,left,190.0,NaN,Neubauer 13 GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/roman-weidenfe...,L1,Borussia Dortmund,750000.0,8000000.0
2,65,Dimitar,Berbatov,Dimitar Berbatov,2015,1091,dimitar-berbatov,Bulgaria,Blagoevgrad,Bulgaria,...,NaN,NaN,NaN,CSKA-AS-23 Ltd.,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/dimitar-berbat...,GR1,Panthessalonikios Athlitikos Omilos Konstantin...,1000000.0,34500000.0
3,77,NaN,Lúcio,Lúcio,2012,506,lucio,Brazil,Brasília,Brazil,...,NaN,NaN,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/lucio/profil/s...,IT1,Juventus Football Club,200000.0,24500000.0
4,80,Tom,Starke,Tom Starke,2017,27,tom-starke,East Germany (GDR),Freital,Germany,...,right,194.0,NaN,IFM,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/tom-starke/pro...,L1,FC Bayern München,100000.0,3000000.0


In [69]:
# Borramos columnas que consideramos irrelevantes o por gran cantidad de datos faltantes

df_players.drop(columns= ['first_name', 'last_name', 'contract_expiration_date', 'agent_name', 'image_url', 'url'], inplace= True)

In [70]:
# rellenamos valores nan

df_players['country_of_birth'].fillna('None', inplace= True)
df_players['city_of_birth'].fillna('None', inplace= True)
df_players['country_of_citizenship'].fillna('None', inplace= True)
df_players['date_of_birth'].fillna(pd.NaT, inplace= True)
df_players['sub_position'].fillna('None', inplace= True)
df_players['foot'].fillna('None', inplace= True)

df_players['height_in_cm'].fillna(0.0, inplace= True)
df_players['market_value_in_eur'].fillna(0.0, inplace= True)
df_players['highest_market_value_in_eur'].fillna(0.0, inplace= True)

In [71]:
# Transformamos el tipo de dato de la columna date_of_birth por datetime

df_players['date_of_birth'] = pd.to_datetime(df_players['date_of_birth'])

In [72]:
df_players['height_in_cm'].unique()

array([184., 190.,   0., 194., 179., 193., 180., 167., 176., 183., 187.,
       188., 196., 195., 191., 178., 174., 182., 185., 175., 189., 172.,
       173., 181., 170., 186., 199., 169., 192., 168., 198., 177., 166.,
       171., 197., 201., 164., 200., 165., 205., 159., 161., 203., 160.,
       207., 163., 162., 204., 202., 206.,  18.])

In [73]:
df_players[df_players['height_in_cm'] == 18.0]

,player_id,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,date_of_birth,sub_position,position,foot,height_in_cm,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
25630,592398,Aïssa Boudechicha,2021,40,aissa-boudechicha,None,Medjana,Algeria,2000-04-13,Left-Back,Defender,left,18.0,FR1,FC Girondins Bordeaux,150000.0,150000.0
26501,628490,Genar Fornés,2021,2687,genar-fornes,Spain,Barcelona,Spain,2000-01-17,Left-Back,Defender,left,18.0,ES1,Cádiz Club de Fútbol S.A.D,150000.0,150000.0


In [74]:
df_players['height_in_cm'].loc[[25630,26501]] = 0.0

In [75]:
df_players['height_in_cm'] = df_players['height_in_cm'].astype(int)

Notas

* los valores 0.0 de las columnas 'height_in_cm', 'market_value_in_eur' y 'highest_market_value_in_eur' significan que no hay dato
* el valor NaT en la columna date significa que no hay dato
* se cambió ciertos valores (los que valen 18.) de 'height_in_cm' a 0.0 por su incongruencia, se los consideró como errores

# <h1 align= 'center'><u>Player Valuations</u></h1>

## Informe Preliminar

### General

In [76]:
r.verificar_tipo_datos(df_player_valuations)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,player_id,[<class 'int'>],100.0,0.0,0
1,date,[<class 'str'>],100.0,0.0,0
2,market_value_in_eur,[<class 'int'>],100.0,0.0,0
3,current_club_id,[<class 'int'>],100.0,0.0,0
4,player_club_domestic_competition_id,[<class 'str'>],100.0,0.0,0


In [77]:
r.informe_dataframe(df_player_valuations)

INFORME PRELIMINAR SOBRE CARACTERISTICAS DEL DATASET:

--Dimensiones del DataFrame--
Filas: 467137
Columnas: 5

--Numero de datos--
2335685

--Filas y Columnas--
Filas: muestra de indices-------> [0, 1, 2, 3, 4]  -----> Desde 0  Hasta 467136
Columnas: ['player_id', 'date', 'market_value_in_eur', 'current_club_id', 'player_club_domestic_competition_id']

--Tipo de columnas--
player_id                               int64
date                                   object
market_value_in_eur                     int64
current_club_id                         int64
player_club_domestic_competition_id    object
dtype: object
--Cantidad de registros duplicados--
0



'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~oo~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

### Por columna

In [78]:
for columna in df_player_valuations.columns:
    r.informe_columna(df_player_valuations, columna)

INFORME PRELIMINAR SOBRE LA COLUMNA/FEATURE player_id:

--Numero de datos nulos--
0

--Valores unicos en la columna--
Ej: [405973 342216   3132   6893     10]  -----> Desde 405973  Hasta 983817

--Estadisticos Principales de la columna--
Media: 205933.01
Desviacion Estandar: 187666.52
Primer cuartil: 57091.0
Mediana: 151213.0
Tercer cuartil: 301758.0

--Valores extremos--
Valor maximo: 1225269
Valor minimo: 10

--Distribucion de frecuencias (primeros valores con mayor cantidad de frecuencias)--
 player_id
39153    52
35664    50
44162    50
Name: count, dtype: int64

--Valor maximo y minimo--
Maximo: 1225269
Minimo: 10

------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------
INFORME PRELIMINAR SOBRE LA COLUMNA/FEATURE date:

--Numero de datos nulos--
0

--Cantidad de valores unicos en la columna--
5145


## ETL

In [79]:
df_player_valuations.head()

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id
0,405973,2000-01-20,150000,3057,BE1
1,342216,2001-07-20,100000,1241,SC1
2,3132,2003-12-09,400000,126,TR1
3,6893,2003-12-15,900000,984,GB1
4,10,2004-10-04,7000000,398,IT1


In [80]:
# Transformamos el tipo de dato de la columna date_of_birth por datetime

df_player_valuations['date'] = pd.to_datetime(df_player_valuations['date'])

# Carga de datos

In [81]:
# # creamos lista de datafra,es y de etiquetas para su posterior guardado como archivos .csv
dfs = [df_appearances,df_club_games, df_clubs, df_competitions, df_game_events, df_game_lineups, df_games, df_player_valuations, df_players]
etiquetas = ['appearances', 'club_games', 'clubs', 'competitions', 'game_events', 'game_lineups', 'games', 'player_valuations', 'players']

In [97]:
# guardamos como archivos .csv
for dataframe, nombre in zip(dfs,etiquetas):
  df = dataframe
  df.to_csv(f'/content/drive/MyDrive/NoCountry_rendimiento_deportivo/data_normalizada/{nombre}.csv', index= False, encoding= 'utf-8')

In [99]:
# guardamos como archivos .parquet
for dataframe, nombre in zip(dfs,etiquetas):
  df= dataframe
  archivo = f'/content/drive/MyDrive/NoCountry_rendimiento_deportivo/data_normalizada/{nombre}.parquet'
  pq.write_table(pa.Table.from_pandas(df), archivo)
